In [ ]:

import pandas as pd
import glob
import json
import os
import webbrowser
all_files = glob.glob("tableau_BudgetData*.xlsx") + glob.glob("tableau_tableau_BudgetData*.xlsx")

all_data = {}
all_income = {}
NAME_MAPPINGS = {
    # תת-תחום mappings (שם רמה 2)
    'ביטוח לאומי': 'הקצבות ביטוח לאומי',
    'העברות ביטוח לאומי': 'הקצבות ביטוח לאומי',
    # משרד התחבורה - שם תת-תחום השתנה ב-2019
    'פיתוח התחבורה': 'תחבורה',
}

def normalize_name(name):
    """Normalize budget item names that changed across years"""
    if pd.isna(name):
        return name
    return NAME_MAPPINGS.get(str(name), str(name))
for filename in all_files:
        print(f"טוען {filename}...")
        df = pd.read_excel(filename)

        # חילוץ שנה
        year_str = ''.join(filter(str.isdigit, filename))[:4]
        year = int(year_str) if year_str else 0

        if year < 2015 or year > 2024:
            continue

        # יצירת מבנה נתונים להיררכיה
        hierarchy_cols = ['שם רמה 1', 'שם רמה 2', 'שם סעיף', 'שם תחום', 'שם תקנה', 'שם מיון רמה 1']

        # --- עיבוד הכנסות ---
        income_items = []
        if 'הוצאה/הכנסה' in df.columns:
            income_df = df[df['הוצאה/הכנסה'] == 'הכנסה']
            
            if 'סוג תקציב' in income_df.columns:
                income_df = income_df[income_df['סוג תקציב'] == 'ביצוע']
            
            # סינון שורות שבהן שם רמה 1 הוא "הכנסות" (הכנסות מדינה כלליות ולא של משרדים)
            if 'שם רמה 1' in income_df.columns:
                income_df = income_df[income_df['שם רמה 1'] != 'הכנסות']

            income_df['הוצאה נטו'] = pd.to_numeric(income_df['הוצאה נטו'], errors='coerce').fillna(0)
            
            for _, row in income_df.iterrows():
                path = []
                for col in hierarchy_cols:
                    raw_val = row.get(col, '')
                    if pd.notna(raw_val):
                        path.append(normalize_name(str(raw_val)))
                    else:
                        path.append('')
                
                value = row['הוצאה נטו']
                
                # הכנסות יכולות להיות חיוביות או שליליות, ניקח את הערך המוחלט אם זה שלילי אבל בדרך כלל זה חיובי בטור הזה
                # אם הערך הוא 0 נדלג
                if value != 0 and path[0]:
                    income_items.append({
                        'path': path,
                        'value': float(value)
                    })
        
        all_income[year] = income_items

        # --- עיבוד הוצאות ---
        if 'הוצאה/הכנסה' in df.columns:
            df = df[df['הוצאה/הכנסה'] == 'הוצאה']

        if 'סוג תקציב' in df.columns:
            df = df[df['סוג תקציב'] == 'ביצוע']

        # סינון קוד רמה 2 = 62 (החזרי חוב קרן) ו-35
        if 'קוד רמה 2' in df.columns:
            df = df[~df['קוד רמה 2'].isin([62, 35])]

        df = df[~df['קוד תחום'].isin([89, 0,91,93,94,95,98])]
        # המרת הוצאה נטו למספר
        df['הוצאה נטו'] = pd.to_numeric(df['הוצאה נטו'], errors='coerce').fillna(0)
        df = df[df['הוצאה נטו'] > 0]
print("done")




טוען tableau_BudgetData2022.xlsx...
טוען tableau_BudgetData2018.xlsx...
טוען tableau_BudgetData2019.xlsx...
טוען tableau_BudgetData2023.xlsx...
טוען tableau_BudgetData2015.xlsx...
טוען tableau_BudgetData2016.xlsx...
טוען tableau_BudgetData2020.xlsx...
טוען tableau_BudgetData2024-update.xlsx...
טוען tableau_BudgetData2017.xlsx...
טוען tableau_tableau_BudgetData2021.xlsx...


In [22]:
df['הוצאה נטו'].sum()

np.float64(540014300.799)